# user change time variable in input

In [10]:
def ROIData(data_folder):

    import numpy as np
    import os, re, csv, tkinter, shutil
    import pandas as pd
    from itertools import compress
    from InROI import InROI
    from tkinter import filedialog
    from openpyxl import load_workbook
    import xlsxwriter

    print('ROIData function BEGIN ------')
    tmin = [0,180,1260]
    tmax = [180,1260,1440]
    flag = input('Use default time period? \nIf yes, input 0\nIf no, input time period:\nInput Sample:0,180,180,1260,1260,1440\n')
    if flag != 0:
        tmin  = flag.split(',')
        tmin = list(map(int,tmin))      
        tmax = flag.split(',')
        tmax = list(map(int,tmax))
    del tmin[1::2]
    del tmax[0::2]
    print('time start for each period:',tmin)
    print('time end for each period:',tmax)
    if flag == '0':
        tmin = [0,180,1260]
        tmax = [180,1260,1440]
        print('use default time set')
        
    data_f = 'C:/Users/ASMLabUser1/Desktop/SAZA_2018_1/%s/FishData' % data_folder
    ROI_folder = 'C:/Users/ASMLabUser1/Desktop/SAZA_2018_1/%s' % data_folder
    # Define function to sort .csv files in order
    _nsre = re.compile('([0-9]+)')
    def natural_sort_key(s):
        return [int(text) if text.isdigit() else text.lower()
                for text in re.split(_nsre, s)]

    csvfiles = []
    for File in os.listdir(data_f):
        if File.endswith('.csv'):
            csvfiles.append(os.path.join(data_f, File))
    csvfiles.sort(key=natural_sort_key)
    data = pd.DataFrame(columns = ['Fish Number', 'TIn','TOut','BoundaryCrossings',\
                                   'MeanVIn','MeanVOut','StdVIn','StdVOut','ENTRIES',\
                                   'Mean T per entry','% time in record','sanity'])

    ## define var
    filename = np.array(['ROI_pre_left','ROI_mid_left','ROI_post_left','ROI_pre_right','ROI_mid_right','ROI_post_right'])
    filename = filename.reshape(2,3)
    ROI_box = [np.array([[0,50], [0,75], [15,50], [15,75]]),np.array([[15,50],[15,75],[30,50],[30,75]])]
    # cal and make file
    if os.path.exists(ROI_folder + '/ROI.xlsx'):
        os.remove(ROI_folder+'/ROI.xlsx')
    # left -> right

    with pd.ExcelWriter(ROI_folder + '/ROI.xlsx',engine = 'xlsxwriter') as writer:
        for m in range(2):
            Coordinates = ROI_box[m]
            for n in range (len(tmin)):
                TMin, TMax = tmin[n], tmax[n]
                for i in range(1, len(csvfiles)+1):
                    print('Fish..'+str(i))
                    # Extract time spent
                    File = pd.read_csv(csvfiles[i-1])
                    cnames = File.columns.tolist()
                    time = File['T']
                    index = 'Fish %s' % i

                    TMin1 = round(File['Sampling_Rate'][0]*TMin)
                    TMax1 = round(File['Sampling_Rate'][0]*TMax)
                    data_in, data_out = InROI(File, Coordinates, TMin1, TMax1)

                    TIn = 0
                    TOut = 0
                    Changes = 0
                    entry = 0
                    for j in range(1, len(data_in)):
                        if data_in[j-1] != data_in[j]:
                            Changes += 1
                    VIn=list(compress(File['V'],data_in[:-1]))
                    VOut=list(compress(File['V'],data_out[:-1]))
                    if VIn:
                        if np.isnan(VIn[0]):
                            del VIn[0]
                    if VOut:
                        if np.isnan(VOut[0]):
                            del VOut[0]
                    dT = File['dT'][1:]
                    TIn += sum(list(compress(dT,data_in[:-1])))
                    TOut += sum(list(compress(dT,data_out[:-1])))
                    sanity = TIn+TOut
                    perc_time_rec = round((TIn/sanity)*100,4)
                    entry = int(Changes/2)
                    if entry ==0:
                        Mean_t_per_entry= 'No entry'
                    else:
                        Mean_t_per_entry = round(TIn/entry,5)

                    # convert to python 3.4: both mean and sd functions exist in statistics module
                    data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\
                            'MeanVIn':np.mean(VIn), 'MeanVOut':np.mean(VOut),\
                            'StdVIn':np.std(VIn), 'StdVOut':np.std(VOut),'ENTRIES':entry,\
                            'Mean T per entry': Mean_t_per_entry,'% time in record':perc_time_rec,\
                            'sanity':sanity}, ignore_index=True)


                #if not os.path.exists(excel_name):

                data.to_excel(writer, sheet_name=filename[m][n], index = False)
                worksheet = writer.sheets[filename[m][n]]
                worksheet.set_column('A:D',10)
                worksheet.set_column('D:L',13)
                data.drop(data.index, inplace = True)


# user change time within code

In [13]:
def ROIData(data_folder):

    import numpy as np
    import os, re, csv, tkinter, shutil
    import pandas as pd
    from itertools import compress
    from InROI import InROI
    from tkinter import filedialog
    from openpyxl import load_workbook
    import xlsxwriter

    print('ROIData function BEGIN ------')
    tmin = [0,180,1260]
    tmax = [180,1260,1440]
    
    data_f = 'C:/Users/ASMLabUser1/Desktop/SAZA_2018_1/%s/FishData' % data_folder
    ROI_folder = 'C:/Users/ASMLabUser1/Desktop/SAZA_2018_1/%s' % data_folder
    # Define function to sort .csv files in order
    _nsre = re.compile('([0-9]+)')
    def natural_sort_key(s):
        return [int(text) if text.isdigit() else text.lower()
                for text in re.split(_nsre, s)]

    csvfiles = []
    for File in os.listdir(data_f):
        if File.endswith('.csv'):
            csvfiles.append(os.path.join(data_f, File))
    csvfiles.sort(key=natural_sort_key)
    data = pd.DataFrame(columns = ['Fish Number', 'TIn','TOut','BoundaryCrossings',\
                                   'MeanVIn','MeanVOut','StdVIn','StdVOut','ENTRIES',\
                                   'Mean T per entry','% time in record','sanity'])

    ## define var
    filename = np.array(['ROI_pre_left','ROI_mid_left','ROI_post_left','ROI_pre_right','ROI_mid_right','ROI_post_right'])
    filename = filename.reshape(2,3)
    ROI_box = [np.array([[0,50], [0,75], [15,50], [15,75]]),np.array([[15,50],[15,75],[30,50],[30,75]])]
    # cal and make file
    if os.path.exists(ROI_folder + '/ROI.xlsx'):
        os.remove(ROI_folder+'/ROI.xlsx')
    # left -> right

    with pd.ExcelWriter(ROI_folder + '/ROI.xlsx',engine = 'xlsxwriter') as writer:
        for m in range(2):
            Coordinates = ROI_box[m]
            for n in range (len(tmin)):
                TMin, TMax = tmin[n], tmax[n]
                for i in range(1, len(csvfiles)+1):
                    print('Fish..'+str(i))
                    # Extract time spent
                    File = pd.read_csv(csvfiles[i-1])
                    cnames = File.columns.tolist()
                    time = File['T']
                    index = 'Fish %s' % i

                    TMin1 = round(File['Sampling_Rate'][0]*TMin)
                    TMax1 = round(File['Sampling_Rate'][0]*TMax)
                    data_in, data_out = InROI(File, Coordinates, TMin1, TMax1)

                    TIn = 0
                    TOut = 0
                    Changes = 0
                    entry = 0
                    for j in range(1, len(data_in)):
                        if data_in[j-1] != data_in[j]:
                            Changes += 1
                    VIn=list(compress(File['V'],data_in[:-1]))
                    VOut=list(compress(File['V'],data_out[:-1]))
                    if VIn:
                        if np.isnan(VIn[0]):
                            del VIn[0]
                    if VOut:
                        if np.isnan(VOut[0]):
                            del VOut[0]
                    dT = File['dT'][1:]
                    TIn += sum(list(compress(dT,data_in[:-1])))
                    TOut += sum(list(compress(dT,data_out[:-1])))
                    sanity = TIn+TOut
                    perc_time_rec = round((TIn/sanity)*100,4)
                    entry = int(Changes/2)
                    if entry ==0:
                        Mean_t_per_entry= 'No entry'
                    else:
                        Mean_t_per_entry = round(TIn/entry,5)

                    # convert to python 3.4: both mean and sd functions exist in statistics module
                    data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\
                            'MeanVIn':np.mean(VIn), 'MeanVOut':np.mean(VOut),\
                            'StdVIn':np.std(VIn), 'StdVOut':np.std(VOut),'ENTRIES':entry,\
                            'Mean T per entry': Mean_t_per_entry,'% time in record':perc_time_rec,\
                            'sanity':sanity}, ignore_index=True)


                #if not os.path.exists(excel_name):

                data.to_excel(writer, sheet_name=filename[m][n], index = False)
                worksheet = writer.sheets[filename[m][n]]
                worksheet.set_column('A:D',10)
                worksheet.set_column('D:L',13)
                data.drop(data.index, inplace = True)


In [14]:
ROIData('tdms')

ROIData function BEGIN ------
Fish..1
Fish..2


C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\890838338.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\
C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\890838338.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\


Fish..3
Fish..4


C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\890838338.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\
C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\890838338.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\


Fish..5
Fish..6


...

C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\890838338.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\
C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\890838338.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\


Fish..17
Fish..18


C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\890838338.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\
C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\890838338.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\


Fish..19
Fish..20


C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\890838338.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\


In [9]:
ROIData('test')

ROIData function BEGIN ------
Use default time period? 
If yes, input 0
If no, input time period:
Input Sample:0,180,180,1260,1260,1440
0
time start for each period: [0]
time end for each period: []
use default time set
Fish..1
Fish..2


C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\947656504.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\
C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\947656504.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\


Fish..3
Fish..4


C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\947656504.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\
C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\947656504.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\


Fish..5
Fish..6


...

C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\947656504.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\
C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\947656504.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\


Fish..3
Fish..4


C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\947656504.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\
C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\947656504.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\


Fish..5
Fish..6


C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\947656504.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\


# ORIGINAL function

In [12]:
def ROIData():

    import numpy as np
    import os, re, csv, tkinter, shutil
    import pandas as pd
    from itertools import compress
    from InROI import InROI
    from tkinter import filedialog
    from openpyxl import load_workbook

    data_f = input("Input name of folder containing fish data: ")
    data_folder = 'C:/Users/ASMLabUser1/Desktop/SAZA_2018_1/%s/FishData' % data_f
    ROI_folder = 'C:/Users/ASMLabUser1/Desktop/SAZA_2018_1/%s' % data_f
    # Define function to sort .csv files in order
    _nsre = re.compile('([0-9]+)')
    def natural_sort_key(s):
        return [int(text) if text.isdigit() else text.lower()
                for text in re.split(_nsre, s)]

    csvfiles = []
    for File in os.listdir(data_folder):
        if File.endswith('.csv'):
            csvfiles.append(os.path.join(data_folder, File))
    csvfiles.sort(key=natural_sort_key)
    data = pd.DataFrame(columns = ['Fish Number', 'TIn','TOut','BoundaryCrossings',\
                                   'MeanVIn','MeanVOut','StdVIn','StdVOut'])

    ## define var
    filename = np.array(['ROI_pre_left','ROI_mid_left','ROI_post_left','ROI_pre_right','ROI_mid_right','ROI_post_right'])
    filename = filename.reshape(2,3)
    excel_name = 'ROI_data1.xlsx'
    # Coordinates = np.array([[15,50],[15,75],[30,50],[30,75]])

    tmin = [0,180,1260]
    tmax = [180,1260,1440]
    # left and right
    ROI_box = [np.array([[0,50], [0,75], [15,50], [15,75]]),np.array([[15,50],[15,75],[30,50],[30,75]])]


    # cal and make file
    if os.path.exists(ROI_folder + '/ROI.xlsx'):
        os.remove(ROI_folder+'/ROI.xlsx')
    # left -> right

    with pd.ExcelWriter(data_f + '/ROI.xlsx') as writer:
        for m in range(2):
            Coordinates = ROI_box[m]
            for n in range (len(tmin)):
                TMin, TMax = tmin[n], tmax[n]
                for i in range(1, len(csvfiles)+1):
                    print('Fish..'+str(i))
                    # Extract time spent
                    File = pd.read_csv(csvfiles[i-1])
                    cnames = File.columns.tolist()
                    time = File['T']
                    index = 'Fish %s' % i

                    TMin1 = round(File['Sampling_Rate'][0]*TMin)
                    TMax1 = round(File['Sampling_Rate'][0]*TMax)
                    data_in, data_out = InROI(File, Coordinates, TMin1, TMax1)

                    TIn = 0
                    TOut = 0
                    Changes = 0
                    for j in range(1, len(data_in)):
                        if data_in[j-1] != data_in[j]:
                            Changes += 1
                    VIn=list(compress(File['V'],data_in[:-1]))
                    VOut=list(compress(File['V'],data_out[:-1]))
                    if VIn:
                        if np.isnan(VIn[0]):
                            del VIn[0]
                    if VOut:
                        if np.isnan(VOut[0]):
                            del VOut[0]
                    dT = File['dT'][1:]
                    TIn += sum(list(compress(dT,data_in[:-1])))
                    TOut += sum(list(compress(dT,data_out[:-1])))

                    # convert to python 3.4: both mean and sd functions exist in statistics module
                    data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\
                            'MeanVIn':np.mean(VIn), 'MeanVOut':np.mean(VOut),\
                            'StdVIn':np.std(VIn), 'StdVOut':np.std(VOut)}, ignore_index=True)


                #if not os.path.exists(excel_name):

                data.to_excel(writer, sheet_name=filename[m][n], index = False)
                print(m,n)
                data.drop(data.index, inplace = True)


In [11]:
ROIData('roi')

ROIData function BEGIN ------
Use default time period? 
If yes, input 0
If no, input time period:
Input Sample:0,180,180,1260,1260,1440
0
time start for each period: [0]
time end for each period: []
use default time set
Fish..1


C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\947656504.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\
C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\947656504.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\


Fish..2
Fish..3


C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\947656504.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\
C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\947656504.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\


Fish..4
Fish..5


...

C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\947656504.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\
C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\947656504.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\


Fish..3
Fish..4


C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\947656504.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\
C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\947656504.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\


Fish..5
Fish..6


C:\Users\ASMLabUser1\AppData\Local\Temp\ipykernel_10784\947656504.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'Fish Number': index, 'TIn':TIn, 'TOut':TOut, 'BoundaryCrossings':Changes,\
